<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this project, we convert addresses into their equivalent latitude and longitude values and use the Foursquare API to explore neighborhoods in Toronto. Using, the **explore** function we get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters using the *k*-means clustering algorithm to complete this task. Finally, the Folium library is used to visualize the neighborhoods in Toronto and their emerging clusters.

## Importing required libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import csv

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#BeautifulSoup for Scraping Web
!pip install beautifulsoup4
from bs4 import BeautifulSoup
!pip install lxml
import lxml
!pip install requests
import requests

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


#### We make a request to the Wikipedia page containing imformation of all the postal codes of Canada using the Requests library. The Beautiful Soup library is then used to parse this response and the title of the page is printed to check if the request if successful, and if the correct information is returned or not.

In [10]:
sourceHTML = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(sourceHTML.content, 'lxml')

table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

#### Creating the Data Frame

In [19]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,Postcode,Borough,Neighbourhood,
1,,M1A,Not assigned,Not assigned,
2,,M2A,Not assigned,Not assigned,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,


#### Cleaning Data

In [38]:
# Dropping rows where Borough isn't assigned.
df4 = df3[df3.Borough != 'Not assigned']
df4.head()

# Grouping rows which have same Postcode
df5 = df4.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df5.head()

#Replacing Neighbourhood value with Borough's name where it isn't defined.
df5['Neighbourhood'] = np.where(df5['Neighbourhood'] == 'Not assigned', df5['Borough'], df5['Neighbourhood'])
df5.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
